# Coupling Generators

In power systems, power plants often produce multiple types of outputs. For example, an Open Cycle Gas Turbine (OCGT) power plant generates electricity and emits CO2. PyPSA allows you to model this using coupled generators, which directly relate to another generator in the network.

In the following, we demonstrate how to represent such a power plant with multiple coupled generators. The example features an OCGT power plant, represented by an electrical generator (primary generator) and a coupled generator accounting for CO2 emissions (secondary generator).

In [ ]:
import numpy as np
import pypsa
import matplotlib.pyplot as plt

Let's start by creating a simple network with a single bus, a load, and a generator. We will use this as a base to add a coupled generator later.

In [ ]:
# Create a new network with 3 snapshots
n = pypsa.Network(snapshots=range(3))

# Add a bus
n.add("Bus", "electricity", unit="MWh", carrier="el")

# Add a non-constant load
n.add("Load", "load", bus="electricity", p_set=[1.5, 2, 2.5])

# Add an extendable primary OCGT generator with electrical output
n.add(
    "Generator",
    "OCGT",
    bus="electricity",
    p_nom=1,
    p_nom_extendable=True,
    marginal_cost=1,
)

## Adding a Coupled Generator

Now that we have a simple network, let's add a coupled generator to model CO2 emissions from the existing gas-fired power plant.

We will introduce a new bus and a store specifically for tracking CO2 emissions and a new generator that is linked to the existing one. The coupling of the output is determined by a coupling coefficient. This coefficient indicates the ratio between the secondary generator (the one being coupled) and the primary generator (the existing one). In our example, this translates to the amount of CO2 produced per unit of electricity generated. Note that keeping the electrical generator as the primary generator simplifies the setup, as you can set all coefficients in units of MW$_{el}$, just like in order example to 

$$
c = 0.6 \frac{t_{CO2}}{MW_{el}}
$$

The attributes that we set for the coupled generator are`p_coupling`, which points to the primary generator, and `p_coupling_coeff`, which determines the coefficient.

In [ ]:
# Add a new bus for CO2 emissions
n.add("Bus", "co2", carrier="co2", unit="t")

# Define the coupling coefficient (600 kg emissions per 1 MWel output from ocgt)
coeff = 0.6

# Add a coupled generator for CO2 emissions
n.add(
    "Generator",
    "OCGT emission",
    bus="co2",
    p_coupling="OCGT",
    p_coupling_coeff=coeff,
    p_nom=np.inf,
)

# Add a store for CO2 emissions
n.add("Store", "co2 atmosphere", bus="co2", e_nom=10)

### Optimization and Results

With the network and coupled generators set up, we can now proceed to optimize the network. The optimization will now consider both the electricity generation and the CO2 emissions.

Let's run the optimization and examine the results.

In [ ]:
n.optimize()

Let's have a look at the power output:

In [ ]:
n.generators_t.p

As you can see the ratio between CO$_2$ output and power output remains constant. Consequently the store at the CO$_2$ but fills up over time.

In [ ]:
n.stores_t.e

## Adding more coupled generators

Following this scheme of coupled generator dispatch, we can define arbitrarily complex systems of coupled generators that model multiple inputs and outputs simultaneously. In the following, we extend the setup to include a coupled generator that models the injection of natural gas into the power plant. We then add two more coupled, extensible generators that model the carbon capture and storage (CCS) process.

Let's start with the raw gas input for the OCGT power plant. We start by adding a new bus with the carrier "Gas" along with a store representing the gas reserve. Then we add a generator "OCGT gas" which is coupled to the existing OCGT power plant. The coupling coefficient is set to the gas input per unit of electricity generated. The "sign" attribute of the new generator is -1, since it takes gas from the storage.  

In [ ]:
n.add("Bus", "gas", carrier="gas", unit="MWh")
n.add("Store", "gas", bus="gas", e_nom=12, e_initial=12)
n.add(
    "Generator",
    "OCGT gas",
    bus="gas",
    p_nom=np.inf,
    sign=-1,
    p_coupling="OCGT",
    p_coupling_coeff=2,
)

Now, we are dealing with a Carbon Capture and Storage (CCS) process. Here’s a simple breakdown of our setup:

We add two expandable generators: “OCGT cc” and “OCGT cs”.

* “OCGT cc” is responsible for extracting CO2 from the exhaust gases of the power plant.
* “OCGT cs” is in charge of storing the extracted CO2 in a storage unit named “co2 stored”.

Let’s understand the relationships:

* “OCGT cc” is connected to the emissions of the OCGT power plant. It has an upper limit, meaning it can only extract up to a certain amount of CO2 from the exhaust gases. Its "sign" attribute is -1, since it extracts co2 from the exhaust gas.
* “OCGT cs” is directly linked to “OCGT cc”. This means whatever amount of CO2 is extracted by “OCGT cc” is the exact amount that “OCGT cs” will store.

With this setup the optimizer can extend the CCS process to the optimal capacity. It will not exceed the capacity of the OCGT power plant, since the "OCGT cc" generator is directly coupled to the emissions of the OCGT power plant.

By limiting the store capacity of the "co2 atmosphere" store, we force the optimization to find a balance between the CO2 emissions and the CCS process.

In [ ]:
n.add(
    "Generator",
    "OCGT cc",
    bus="co2",
    p_coupling="OCGT",
    p_coupling_coeff=coeff,
    p_coupling_sign="<=",
    p_nom_extendable=True,
    capital_cost=1,
    sign=-1,
)

n.add("Bus", "co2 stored", carrier="co2", unit="t")
n.add(
    "Generator",
    "OCGT cs",
    bus="co2 stored",
    p_coupling="OCGT cc",
    p_nom=np.inf,
)
n.add("Store", "co2 stored", bus="co2 stored", e_nom=20)

n.stores.loc["co2 atmosphere", "e_nom"] = 2

We optimize and analyze the results:

In [ ]:
n.optimize()

In [ ]:
def get_unit(df):
    return "[" + df.bus.map(n.buses.unit) + "$_{" + df.bus.map(n.buses.carrier) + "}$]"


def get_labels(df):
    return df.index + " " + get_unit(df)

In [ ]:
n.generators_t.p.plot.bar(title="Generator Input and Output")
plt.legend(get_labels(n.generators))
plt.tight_layout()

In [ ]:
axes = n.stores_t.p.mul(-1).plot.bar(subplots=True)
for ax, unit in zip(axes, get_unit(n.stores)):
    ax.set_ylabel("Dispatch " + unit)
plt.tight_layout()

In [ ]:
# plot the state of charge (SOC)
axes = n.stores_t.e.plot.bar(subplots=True)
for ax, unit in zip(axes, get_unit(n.stores)):
    ax.set_ylabel("SOC " + unit)
plt.tight_layout()